In [2]:
import torch
import kenlm
import ctcdecode
import pickle
import numpy as np

from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor

from datasets import load_from_disk, load_dataset, load_metric, Dataset, concatenate_datasets

In [2]:
tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

model = Wav2Vec2ForCTC.from_pretrained("./wav2vec2-large-xlsr-ger-chris/checkpoint-51000")

In [3]:
vocab_dict = tokenizer.get_vocab()
sort_vocab = sorted((value, key) for (key,value) in vocab_dict.items())
vocab = [x[1].replace("|", " ") if x[1] not in tokenizer.all_special_tokens else "_" for x in sort_vocab]

In [4]:
alpha = 2.5 # LM Weight
beta = 0.0 # LM Usage Reward
word_lm_scorer = ctcdecode.WordKenLMScorer('lm_data/train.arpa', alpha, beta) # use your own kenlm model

decoder = ctcdecode.BeamSearchDecoder(
    vocab,
    num_workers=2,
    beam_width=128,
    scorers=[word_lm_scorer],
    cutoff_prob=np.log(0.000001),
    cutoff_top_n=40
)

found 1gram
found 2gram


In [5]:
res51_full_log = load_from_disk("res51_full_log")

In [6]:
test_sampled = load_from_disk("/media/chris/TheFlash/Master/data/test_sampled")

In [6]:
def infer_lm(batch, word_lm_scorer, vocabulary):
    from ctcdecode.prefix import State
    import numpy as np

    def get_pruned_vocab_indices(log_probs):
        """ Return vocab indices of pruned probabilities of a time step. """

        index_to_prob = [(k, log_probs[k]) for k in range(log_probs.shape[0])]
        index_to_prob = sorted(index_to_prob, key=lambda x: x[1], reverse=True)

        if 40 < len(index_to_prob):
            index_to_prob = index_to_prob[:40]

        if np.log(0.000001) < 1.0:
            filtered = []
            for x in index_to_prob:
                if x[1] >= np.log(0.000001):
                    filtered.append(x)
            index_to_prob = filtered

        return [x[0] for x in index_to_prob]

    def decode(probs):
        # Num time steps
        nT = probs.shape[0]

        # Initialize prefixes
        prefixes = State(
            scorers=[word_lm_scorer],
            size=128
        )

        # Iterate over timesteps
        for t in range(nT):
            step_probs = probs[t]
            pruned_step_probs = get_pruned_vocab_indices(step_probs)

            # Iterate over symbols
            for v in pruned_step_probs:
                symbol = vocabulary[v]
                symbol_prob = step_probs[v]

                # Iterate over prefixes
                for prefix in prefixes:

                    # If there is a blank, we extend the existing prefix
                    if symbol == '_':
                        prefix.add_p_blank(symbol_prob + prefix.score)

                    else:

                        # If the last symbol is repeated
                        # update the existing prefix
                        if symbol == prefix.symbol:
                            p = symbol_prob + prefix.p_non_blank_prev
                            prefix.add_p_non_blank(p)

                        new_prefix = prefixes.get_prefix(prefix, symbol)

                        if new_prefix is not None:
                            p = -np.inf

                            if symbol == prefix.symbol and \
                                    prefix.p_blank_prev > -np.inf:
                                p = prefix.p_blank_prev + symbol_prob

                            elif prefix.symbol != symbol:
                                p = prefix.score + symbol_prob

                            new_prefix.add_p_non_blank(p)

            prefixes.step()

        prefixes.finalize()

        return prefixes.best()

    batch["lm_str"] = decode(np.asarray(batch["lm_raw"]))
    return batch

In [7]:
print(len(res51_full_log))

15588


In [ ]:
from datasets import Dataset
batch_size = 2000
dataset_length = len(res51_full_log)

for i in range(0, dataset_length, batch_size):
    chunk = Dataset.from_dict(res51_full_log[i:i+batch_size])
    chunk = chunk.map(infer_lm, fn_kwargs=dict(word_lm_scorer=word_lm_scorer, vocabulary=vocab), num_proc=6)
    chunk.save_to_disk("lm_res51/full_full_chunks/chunk" + str(i) + "_" + str(i+batch_size))

In [3]:
import glob

res51 = load_from_disk("lm_res51/full_full_chunks/chunk0_2000")

for file_name in glob.iglob("lm_res51/full_full_chunks/chunk*"):
    if(file_name =="lm_res51/full_full_chunks\chunk0_2000"):
        i= 0
        # do nothing
    else:
        print(file_name)
        res_chunk = load_from_disk(file_name)
        res51 = concatenate_datasets([res51, res_chunk])   

lm_res51/full_full_chunks\chunk2000_4000
lm_res51/full_full_chunks\chunk4000_6000
lm_res51/full_full_chunks\chunk6000_8000
lm_res51/full_full_chunks\chunk8000_10000
lm_res51/full_full_chunks\chunk10000_12000
lm_res51/full_full_chunks\chunk12000_14000
lm_res51/full_full_chunks\chunk14000_16000


In [4]:
len(res51)

15588

In [5]:
res51.save_to_disk("lm_full_full")

In [13]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [14]:
show_random_elements(res51.remove_columns(["speech", "sampling_rate", "lm_raw"]))

,lm_str,pred_str,target_text
0,wago aus sieben bürgen die stadt,wagozchi aus sieben bürgen die stadt,rkczi aus siebenbürgen die stadt
1,im laufe der zeit wandelte sich der name zu de,im laufe der zeit wandelte sich der name zu ede,im laufe der zeit wandelte sich der name zu de
2,balzer begann ihre sportliche karriere als sieben kämpferin,balte begann ihre sportliche karriere als siebenkämpferin,balta begann ihre sportliche karriere als siebenkämpferin
3,wir sind doch hier nicht eine f,wir sind doch hier nicht eine fh,wir sind hier doch nicht an der fh
4,sieben,sieben,sieben
5,es ist mit eis und glatte zum rechten warum sie daher besonders vorsichtig,es ist mit eis und glätte zum rechten warum sie daher besonders vorsichlih,es ist mit eis und glätte zu rechnen fahren sie daher besonders vorsichtig
6,wir sind zwillinge,wir sind zwi linge,wir sind zwillinge
7,alle in frankreich geltenden gesetze wurden eingeführt,alle in frankreich geltenden gesetze wurden eingeführt,alle in frankreich geltenden gesetze wurden eingeführt
8,was läuft heute abend im fernsehen,was läuft heute abend im fernsehen,was läuft heute abend im fernsehen
9,er spielte unter anderem hamlet und mack bath,er spielte unter anderem humlet und mack bath,er spielte unter anderem hamlet und macbeth


In [5]:
wer_metric = load_metric("wer")

In [11]:
print("Test WER: {:.3f}".format(wer_metric.compute(predictions=res51["lm_str"], references=res51["target_text"])))

Test WER: 0.148


In [12]:
print("Test WER: {:.3f}".format(wer_metric.compute(predictions=res51["pred_str"], references=res51["target_text"])))

Test WER: 0.171


In [2]:
from datasets import load_from_disk, load_dataset

results51_full = load_from_disk("results51_test")
results51_cut = load_from_disk("results51_cut")
results51_cut_log = load_from_disk("res51_cut_log")
results51_full_log = load_from_disk("res51_full_log")

In [6]:
print("Full Test WER: {:.3f}".format(wer_metric.compute(predictions=results51_full["pred_str"], references=results51_full["target_text"])))
print("Cut Test WER: {:.3f}".format(wer_metric.compute(predictions=results51_cut["pred_str"], references=results51_cut["target_text"])))
print("Cut Log Test WER: {:.3f}".format(wer_metric.compute(predictions=results51_cut_log["pred_str"], references=results51_cut_log["target_text"])))
print("Full Log Test WER: {:.3f}".format(wer_metric.compute(predictions=results51_full_log["pred_str"], references=results51_full_log["target_text"])))

Full Test WER: 0.147
Cut Test WER: 0.148
Cut Log Test WER: 0.148
Full Log Test WER: 0.147


In [16]:
for i in range(10):
    print(i)
    print(res51[i]["pred_str"])
    print(res51[i]["lm_str"])
    print("")
    print(results51_full[i]["pred_str"])
    print(results51_cut[i]["pred_str"])
    print(results51_cut_log[i]["pred_str"])
    print(results51_full_log[i]["pred_str"])
    print("")
    print(results51_full_log[i]["target_text"])

0
sieht euch bitte draußen die schuhe aus
zieht euch bitte draußen die schuhe aus 

zieht euch bitte draußen die schuhe aus
sieht euch bitte draußen die schuhe aus
sieht euch bitte draußen die schuhe aus
zieht euch bitte draußen die schuhe aus

zieht euch bitte draußen die schuhe aus 
1
es romi schwon geworden tet
des komischerweise 

des komtikolnegabentert
es romi schwon geworden tet
es romi schwon geworden tet
des komtikolnegabentert

es kommt zum showdown in gstaad 
2
ihre forterstrecken erschienen den modemagazin wie der wolg ab das basain var ricler
ihre fotostrecken erschienen in modemagazin wie der volk hat das basra regler 

ihre fortestrecken erschienen mit molemagazine wie der wog at das basarwaryclar
ihre forterstrecken erschienen den modemagazin wie der wolg ab das basain var ricler
ihre forterstrecken erschienen den modemagazin wie der wolg ab das basain var ricler
ihre fortestrecken erschienen mit molemagazine wie der wog at das basarwaryclar

ihre fotostrecken erschiene